In [1]:
import pytesseract

text = pytesseract.image_to_string('/Users/bailey/Downloads/data-science/Vergil-OCR-test/images/forbes-demo-eng.jpg')
print(text)

TesseractError: (1, 'Error, cannot read input file /Users/bailey/Downloads/data-science/Vergil-OCR-test/images/forbes-demo-eng.jpg: No such file or directory Error during processing.')

In [ ]:
import io
import requests
import pytesseract
from PIL import Image
response = requests.get("https://d2jaiao3zdxbzm.cloudfront.net/wp-content/uploads/figure-65.png")
# print( type(response) ) # <class 'requests.models.Response'>
img = Image.open(io.BytesIO(response.content))
# print( type(img) ) # <class 'PIL.JpegImagePlugin.JpegImageFile'>
text = pytesseract.image_to_string(img)
print( text )

ModuleNotFoundError: No module named 'requests'

In [ ]:
import pytesseract
from googletrans import Translator
from PIL import Image

translator = Translator()
im = Image.open("/Users/bailey/Downloads/data-science/Vergil-OCR-test/images/uber-das.png")

text = pytesseract.image_to_string(im, lang = 'deu')
print("ORIGINAL\n",text)

translation = translator.translate(text, dest='en')
print("TRANSLATED\n",text)
print(translation)

ORIGINAL
 über das

TRANSLATED
 über das

Translated(src=de, dest=en, text=about the, pronunciation=None, extra_data="{'confiden...")


Notes:
- doesn't read the scanned version of deu.jpg well at all; reads unscanned ver. of jahrenbericht better than the scanned version..
- doesn't pick up on 'uber des' and some of the smaller text.
- bad at latin
    - better at scanned; cannot decipher V as translation horrible as expected

In [125]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

xls = pd.read_excel("/Users/bailey/Downloads/PA-vergil-website/Vergil-OCR-test/PAVergilWebsite_BaileyXu_2024.xlsx", 
                    'Website - Draft (Print info.)',
                    header=0, 
                    skiprows=[0])
xls['Date (Print):'] = xls['Date (Print):'].str.replace('.','')
xls['Publisher (IntArc)'] = None
xls['Language (IntArc)'] = None
xls['Contributor (IntArc)'] = None

for i in range (len(xls)):
    title = str(xls["Title/Description (Print – Original):"][i])[:100]
    date = str(xls["Date (Print):"][i])
    if (title=='[-]'):
        continue
        
    out_link = "https://www.google.com/search?q=" + '+'.join(title.split(' '))
    page = requests.get(out_link)
    soup = BeautifulSoup(page.content)

    links = []
    for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        for l in re.split(":(?=http)",link["href"].replace("/url?q=","")):
            links.append(l)

    links = [link for link in links if ((link.find("archive.org/details/") > 0) or (link.find("search.worldcat.org") > 0))]
    clean_links = []
    for link in links:
        if (link.find("search.worldcat.org") > 0):
            clean_links.append(link.split("&")[0])
        else:
            clean_links.append(link)
    # print(clean_links)
    
    for link in clean_links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content)
        # if (link.find("archive.org/details/") > 0):
        #     # check publication year
        #     spans = soup.find('span', {'itemprop' : 'datePublished'})
        #     link_date = spans.get_text()
        #     if (date!=link_date):
        #         print(f"Mismatched date for {title} {link}")
        #         continue
            
        #     # Get Publisher
        #     try:
        #         spans = soup.find_all('span', {'itemprop' : 'publisher'})
        #         publisher = spans[0].get_text()
        #         print(publisher)
        #         xls.loc[i, 'Publisher (IntArc)'] = publisher
        #     except:
        #         print(f"Publisher Failed for {title} {link}")
        #         pass
                
        #     # Get Language
        #     try:
        #         spans = soup.find_all('a',href=True)
        #         for span in spans:
        #             if (span['href'].find("28language%") > 0):
        #                 lang = span.next
        #                 print(lang)
        #                 xls.loc[i, 'Language (IntArc)'] = lang
        #                 break
        #     except:
        #         print(f"Language Failed for {title} {link}")
        #         pass
                
        #     # Get Contributor
        #     try:
        #         spans = soup.find('dl', {'class' : 'metadata-definition'})
        #         if (spans.get_text().find("by") > 0):
        #             cont = spans.get_text().strip(' by\n')
        #             print(cont)
        #             xls.loc[i, 'Contributor (IntArc)'] = cont
        #     except:
        #         print(f"Contributor Failed for {title} {link}")
        #         pass
            
        #     # Get pdf link
        # el
        if (link.find("search.worldcat.org") > 0):
            code = link.find("oclc/") + 5
            code = link[code:]
            # print(code)
            
            # check publication year + get language
            # try:
            #     search='format-'+code
            #     texts = soup.find('span', {'data-testid' : search}).parent
            #     spans = texts.find_all('span')
            #     lang = spans[-1].next
    
            #     texts = str(texts)
            #     year = re.findall(r'\d+', texts[-12:])
            #     year = int(year[0])
            # except:
            #     pass

            # # Get authors
            # search='author-'+code
            # spans = soup.find_all('span', {'aria-labelledby' : search})
            # for span in spans:
            #     temp = span.find_all('a')
                
            #     for l in temp:
            #         # print(l.next)

            # Get publisher
            search='publisher-'+code
            spans = soup.find_all('span', {'data-testid' : search})
            print(spans.next)
    break
    print(xls.loc[i, ['Publisher (IntArc)','Language (IntArc)','Contributor (IntArc)']])            

<>:26: SyntaxWarning: invalid escape sequence '\?'
<>:26: SyntaxWarning: invalid escape sequence '\?'
/var/folders/st/vmdh21yx7z921b5950g1xycw0000gn/T/ipykernel_66059/418801127.py:26: SyntaxWarning: invalid escape sequence '\?'
  for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
/var/folders/st/vmdh21yx7z921b5950g1xycw0000gn/T/ipykernel_66059/418801127.py:26: SyntaxWarning: invalid escape sequence '\?'
  for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):


AttributeError: ResultSet object has no attribute 'next'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [117]:
xls['Publisher (IntArc)']

0       None
1       None
2       None
3       None
4       None
        ... 
1461    None
1462    None
1463    None
1464    None
1465    None
Name: Publisher (IntArc), Length: 1466, dtype: object

In [118]:
xls['Contributor (IntArc)']

0       None
1       None
2       None
3       None
4       None
        ... 
1461    None
1462    None
1463    None
1464    None
1465    None
Name: Contributor (IntArc), Length: 1466, dtype: object

In [119]:
xls['Language (IntArc)']

0       None
1       None
2       None
3       None
4       None
        ... 
1461    None
1462    None
1463    None
1464    None
1465    None
Name: Language (IntArc), Length: 1466, dtype: object